In [1]:
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables
load_dotenv()


# Build correct local path (relative to project root)
local_path = Path("data/raw/Binance_BTCUSDT_1h.csv")

# Ensure it exists
if not local_path.exists():
    raise FileNotFoundError(f"File not found: {local_path.resolve()}")

# Load CSV
print(f"Loading local file: {local_path}")
df = pd.read_csv(local_path, skiprows=1)

Loading local file: data\raw\Binance_BTCUSDT_1h.csv


In [2]:
# Convert Date column
df['Date'] = pd.to_datetime(df['Date'], errors="coerce", format="mixed")

print(df.dtypes)
print(df['Date'].head())

Unix                    int64
Date           datetime64[ns]
Symbol                 object
Open                  float64
High                  float64
Low                   float64
Close                 float64
Volume BTC            float64
Volume USDT           float64
tradecount              int64
dtype: object
0   2025-08-22 23:00:00
1   2025-08-22 22:00:00
2   2025-08-22 21:00:00
3   2025-08-22 20:00:00
4   2025-08-22 19:00:00
Name: Date, dtype: datetime64[ns]


In [3]:
duplicates = df.duplicated()
print("Total duplicate rows:", duplicates.sum())

Total duplicate rows: 0


In [4]:
print("Duplicate timestamps:", df['Date'].duplicated().sum())

Duplicate timestamps: 506


In [5]:
# Count how many times each timestamp appears
timestamp_counts = df['Date'].value_counts().sort_values(ascending=False)

# Show only duplicates (timestamps with more than 1 occurrence)
duplicates_fixed = timestamp_counts[timestamp_counts > 1]

print("Number of duplicated timestamps:", len(duplicates_fixed))
print(duplicates_fixed.head(10))  # show top 10 duplicates

Number of duplicated timestamps: 506
Date
2025-03-03 21:00:00    2
2025-03-07 10:00:00    2
2025-03-07 23:00:00    2
2025-03-07 22:00:00    2
2025-03-07 21:00:00    2
2025-03-07 20:00:00    2
2025-03-07 19:00:00    2
2025-03-07 18:00:00    2
2025-03-07 17:00:00    2
2025-03-07 16:00:00    2
Name: count, dtype: int64


In [6]:
# Find all duplicates by Date
dupes = df[df['Date'].duplicated(keep=False)].sort_values('Date')

# Check if all values in these duplicates are the same
all_equal = dupes.groupby('Date').nunique().max(axis=1).eq(1).all()

print("Are all duplicate rows identical?", all_equal)

Are all duplicate rows identical? False


In [7]:
df = df.groupby('Date', as_index=False).agg({
    'Symbol': 'first',     # same for all rows
    'Open': 'mean',        # average price at open
    'High': 'max',         # take the maximum high
    'Low': 'min',          # take the minimum low
    'Close': 'last',       # take the last close
    'Volume BTC': 'sum',   # sum up the volumes
    'Volume USDT': 'sum',
    'tradecount': 'sum'    # total number of trades
})

print("Duplicate timestamps left:", df['Date'].duplicated().sum())

Duplicate timestamps left: 0


# Display the information about the Dataset

In [8]:
# General info about columns, types, non-null counts
print(df.info())

# Basic statistics for numeric columns
print(df.describe().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69631 entries, 0 to 69630
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         69631 non-null  datetime64[ns]
 1   Symbol       69631 non-null  object        
 2   Open         69631 non-null  float64       
 3   High         69631 non-null  float64       
 4   Low          69631 non-null  float64       
 5   Close        69631 non-null  float64       
 6   Volume BTC   69631 non-null  float64       
 7   Volume USDT  69631 non-null  float64       
 8   tradecount   69631 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 4.8+ MB
None
               count                           mean                  min  \
Date           69631  2021-08-11 11:05:11.666871552  2017-08-17 04:00:00   
Open         69631.0                   32372.715203               2870.9   
High         69631.0                   

In [10]:
# Display the first and last 5 rows
print(df.head())
print(df.tail())

                 Date   Symbol     Open     High      Low    Close  \
0 2017-08-17 04:00:00  BTCUSDT  4261.48  4313.62  4261.32  4308.83   
1 2017-08-17 05:00:00  BTCUSDT  4308.83  4328.69  4291.37  4315.32   
2 2017-08-17 06:00:00  BTCUSDT  4330.29  4345.45  4309.37  4324.35   
3 2017-08-17 07:00:00  BTCUSDT  4316.62  4349.99  4287.41  4349.99   
4 2017-08-17 08:00:00  BTCUSDT  4333.32  4377.85  4333.32  4360.69   

   Volume BTC    Volume USDT  tradecount  
0   47.181009  202366.138393         171  
1   23.234916  100304.823567         102  
2    7.229691   31282.312670          36  
3    4.443249   19241.058300          25  
4    0.972807    4239.503586          28  
                     Date   Symbol       Open       High        Low  \
69626 2025-08-22 19:00:00  BTCUSDT  116805.27  116914.30  116520.00   
69627 2025-08-22 20:00:00  BTCUSDT  116655.79  117127.44  116570.17   
69628 2025-08-22 21:00:00  BTCUSDT  117052.65  117174.43  116829.34   
69629 2025-08-22 22:00:00  BTCUSDT  1

In [13]:
# Count null values in each column
print(df.isna().sum())

# Check if *any* null exists
print("Any missing values?", df.isna().any().any())

Date           0
Symbol         0
Open           0
High           0
Low            0
Close          0
Volume BTC     0
Volume USDT    0
tradecount     0
dtype: int64
Any missing values? False


In [12]:
import numpy as np

# Extract time features
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month

# Cyclical encoding for hour
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Cyclical encoding for day of week (7 days)
df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

print(df.head())


AttributeError: 'RangeIndex' object has no attribute 'hour'